# Lab 34: LangChain Agents with Tool Calling

## Overview
This lab introduces **LangChain Agents** - intelligent systems that can reason about which tools to use and how to use them to accomplish tasks. Unlike simple chains that follow predefined paths, agents can make decisions dynamically based on the input and available tools.

## Key Concepts
- **Tool Calling Agents**: Agents that can invoke external tools and APIs
- **Agent Executor**: The runtime environment that manages agent execution
- **Agent Memory**: Maintaining conversation history across interactions
- **Dynamic Tool Selection**: How agents choose which tools to use for specific tasks

## Learning Objectives
- Understand the difference between chains and agents
- Create tool-calling agents with external search capabilities
- Implement agent memory for contextual conversations
- Configure agent execution with proper error handling
- Learn agent reasoning patterns and decision-making processes

## What You'll Build
A conversational agent that can:
1. Search the web for current information using Tavily
2. Maintain conversation context across multiple interactions
3. Reason about when and how to use tools
4. Provide informed responses based on retrieved information

In [ ]:
# Import Required Libraries for Agent Creation

# Core LangChain components for building conversational agents
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

# OpenAI integration for language model capabilities
from langchain_openai import ChatOpenAI

# Tavily search tool for real-time web search capabilities
from langchain_community.tools.tavily_search import TavilySearchResults

# Agent creation and execution components
from langchain.agents import create_tool_calling_agent, tool
from langchain.agents import AgentExecutor
from langchain.agents import AgentType, initialize_agent, load_tools

# Environment variable management
import os

# Agent Architecture Overview:
# 1. ChatPromptTemplate: Structures the conversation format for the agent
# 2. MessagesPlaceholder: Handles dynamic insertion of chat history
# 3. ChatMessageHistory: Stores conversation context across interactions
# 4. RunnableWithMessageHistory: Adds memory capabilities to agents
# 5. create_tool_calling_agent: Creates agents that can invoke external tools
# 6. AgentExecutor: Manages the execution loop and tool calling process

In [ ]:
# Configure Tavily Search Tool

# Set up Tavily API credentials for web search functionality
# Tavily provides AI-optimized search results specifically designed for LLM applications
os.environ["TAVILY_API_KEY"] = "API_KEY"  # Replace with your actual API key

# Initialize the Tavily search tool
# TavilySearchResults returns structured search results with:
# - Relevant web content summaries
# - Source URLs for verification
# - Metadata about search quality and relevance
search = TavilySearchResults()

# Why Tavily for Agents?
# - Optimized for AI applications with clean, structured output
# - Reduces hallucination by providing current, factual information
# - Includes source attribution for transparency and verification
# - Designed to work seamlessly with LangChain agent workflows

In [ ]:
# Create Agent Prompt Template

# Design a conversation structure that enables proper agent reasoning
prompt = ChatPromptTemplate.from_messages(
    [
        # System message: Sets the agent's behavior and reasoning approach
        ("system", "You are a helpful assistant. Think step by step before responding."),
        
        # Chat history placeholder: Injects previous conversation context
        # This enables the agent to maintain continuity across interactions
        ("placeholder", "{chat_history}"),
        
        # Human input: The current user query or request
        ("human", "{input}"),
        
        # Agent scratchpad: Space for the agent's reasoning and tool calling process
        # This is where the agent records its thoughts and tool execution results
        ("placeholder", "{agent_scratchpad}")
    ]
)

# Prompt Template Structure Explanation:
# 1. System Message: Establishes agent personality and reasoning approach
# 2. Chat History: Provides context from previous conversation turns
# 3. Human Input: Contains the current user query requiring response
# 4. Agent Scratchpad: Working space for agent reasoning and tool results

# The "step by step" instruction encourages the agent to:
# - Analyze the query carefully
# - Determine if tools are needed
# - Plan the sequence of actions
# - Execute tools systematically
# - Synthesize results into a coherent response

In [ ]:
# Initialize Language Model for Agent

# Configure OpenAI API credentials
import os
os.environ["OPENAI_API_KEY"] = "api_key"  # Replace with your actual API key

# Initialize ChatOpenAI model for agent reasoning
# The LLM serves as the "brain" of the agent, responsible for:
# - Understanding user queries
# - Deciding which tools to use
# - Reasoning about tool outputs
# - Generating final responses
llm = ChatOpenAI()

# Why ChatOpenAI for Agents?
# - Strong reasoning capabilities for tool selection
# - Excellent at following complex prompt instructions
# - Reliable function calling support for tool integration
# - Good balance of speed and intelligence for agent workflows

# Agent LLM Requirements:
# - Function calling support (for tool invocation)
# - Strong reasoning capabilities (for decision making)
# - Instruction following (for prompt adherence)
# - Context management (for conversation continuity)

In [ ]:
# Define Agent Tools

# Create the tools list that the agent can access and use
# Each tool in this list becomes available for the agent to invoke
tools = [search]

# Tool Selection Process:
# 1. Agent analyzes the user query
# 2. Determines if external information is needed
# 3. Selects appropriate tool(s) from available options
# 4. Executes tool with relevant parameters
# 5. Processes tool output for response generation

# Available Tools in This Agent:
# - TavilySearchResults: Web search for current information
#   - Use case: Questions requiring recent data or facts
#   - Input: Search query string
#   - Output: Structured search results with sources

# Tool Architecture Benefits:
# - Modular design: Easy to add/remove tools
# - Automatic integration: Tools work seamlessly with agent logic
# - Standardized interface: All tools follow same calling pattern
# - Error handling: Built-in error management for tool failures

# Expanding Tool Capabilities:
# You can extend this agent by adding more tools:
# - Calculator tools for mathematical operations
# - Database tools for structured data queries
# - API tools for specific service integrations
# - Custom tools for business-specific operations

In [ ]:
# Create Message History for Agent Memory

# Initialize a message history object to store conversation context
# This enables the agent to remember previous interactions
message_history = ChatMessageHistory()

# Memory Architecture:
# - Stores both user messages and agent responses
# - Maintains conversation context across multiple turns
# - Enables reference to previous topics and information
# - Supports follow-up questions and context-dependent queries

# Memory Benefits for Agents:
# 1. Contextual Understanding: Agent can reference previous conversation
# 2. Follow-up Support: Users can ask "tell me more" or "what about X?"
# 3. Efficiency: Avoids re-searching for previously retrieved information
# 4. Natural Conversation: Creates more human-like interaction patterns

# Memory Considerations:
# - Token limits: Long conversations may exceed model context windows
# - Privacy: Sensitive information persists in memory
# - Performance: Large histories may slow down processing
# - Cleanup: Consider clearing history for new conversation topics

# Alternative Memory Options:
# - RedisChatMessageHistory: Persistent storage across sessions
# - FileChatMessageHistory: File-based storage for local persistence
# - Custom memory: Application-specific memory implementations

In [ ]:
# Create Tool-Calling Agent

# Combine LLM, tools, and prompt into a cohesive agent
# This creates the core agent that can reason and use tools
agent = create_tool_calling_agent(llm, tools, prompt)

# Agent Creation Process:
# 1. LLM provides reasoning and decision-making capabilities
# 2. Tools define available external actions and data sources
# 3. Prompt structures the conversation and reasoning format
# 4. Agent combines these components into intelligent workflow

# Tool-Calling Agent Features:
# - Automatic tool selection based on query analysis
# - Structured reasoning process with scratchpad tracking
# - Integration of tool outputs into natural language responses
# - Error handling for tool failures and invalid parameters

# Agent Decision Making:
# The agent follows this reasoning pattern:
# 1. Analyze user input for intent and requirements
# 2. Determine if external tools are needed
# 3. Select appropriate tool(s) and parameters
# 4. Execute tool calls and process results
# 5. Synthesize information into coherent response
# 6. Update conversation memory with interaction

# Agent vs Chain Comparison:
# - Chains: Predetermined sequence of operations
# - Agents: Dynamic decision-making about tool usage
# - Chains: Faster, more predictable execution
# - Agents: More flexible, handles complex multi-step tasks

In [ ]:
# Initialize Agent Executor

# Create the execution environment that manages agent operations
# AgentExecutor handles the complete agent lifecycle and tool orchestration
agent_executor = AgentExecutor(
    agent=agent,           # The reasoning agent created above
    tools=tools,          # Available tools for the agent to use
    verbose=False         # Set to True to see detailed execution logs
)

# Agent Executor Responsibilities:
# 1. Input Processing: Parse and validate user queries
# 2. Agent Invocation: Call the agent with structured input
# 3. Tool Orchestration: Manage tool calls and error handling
# 4. Output Management: Format and return agent responses
# 5. Safety Controls: Implement guardrails and limits

# Execution Flow:
# User Input → Agent Executor → Agent Reasoning → Tool Calls → Response

# Verbose Mode Benefits (set verbose=True to enable):
# - See agent's step-by-step reasoning process
# - Monitor tool selection and execution
# - Debug issues with tool calls or responses
# - Understand agent decision-making patterns

# Agent Executor Configuration Options:
# - max_iterations: Limit agent reasoning steps
# - max_execution_time: Set timeout for agent operations
# - early_stopping_method: Control when agent stops reasoning
# - return_intermediate_steps: Include reasoning steps in output

In [ ]:
# Create Agent with Memory Integration

# Wrap the agent executor with memory capabilities for persistent conversations
agent1 = RunnableWithMessageHistory(
    agent_executor,                    # The core agent executor
    lambda session_id: message_history, # Function returning message history
    input_messages_key="input",        # Key for user input in the message structure
    history_messages_key="chat_history", # Key for conversation history in prompts
)

# Memory Integration Architecture:
# 1. User provides input with session configuration
# 2. System retrieves conversation history for the session
# 3. History gets injected into the prompt template
# 4. Agent processes query with full conversation context
# 5. Response and query are saved to message history

# Session Management:
# - session_id: Unique identifier for conversation threads
# - Multiple sessions: Support parallel conversations
# - Session isolation: Each conversation maintains separate context
# - Session persistence: Conversations survive application restarts

# Message Flow with Memory:
# Input → History Retrieval → Context Injection → Agent Processing → Response + Save

# Memory Configuration Parameters:
# - input_messages_key: Where to find user input in invoke() call
# - history_messages_key: Where to inject history in prompt template
# - output_messages_key: Where to find agent response for storage

# Benefits of Memory-Enabled Agents:
# - Contextual responses that reference previous conversation
# - Support for follow-up questions and clarifications
# - Natural conversation flow with topic continuity
# - Ability to build on previous search results and information

In [ ]:
# Execute Agent with Real-World Query

# Invoke the agent with a current events query that requires web search
# This demonstrates the agent's ability to retrieve and process real-time information
response = agent1.invoke(
    {"input": "When is the ICC Men's T20 2024 World Cup scheduled?"},
    config={"configurable": {"session_id": "session1"}}
)

# Display the agent's response
print("Agent Response:")
print(response['output'])

# Agent Execution Process for This Query:
# 1. Query Analysis: Agent identifies need for current sports information
# 2. Tool Selection: Determines Tavily search is appropriate
# 3. Search Execution: Performs web search for T20 World Cup 2024 schedule
# 4. Result Processing: Extracts relevant dates and information
# 5. Response Generation: Synthesizes findings into natural language answer

# Why This Query Tests Agent Capabilities:
# - Requires current information not in training data
# - Tests tool selection logic (search vs other options)
# - Demonstrates information extraction from search results
# - Shows natural language synthesis of retrieved data

# Session Configuration:
# - session_id "session1": Creates persistent conversation thread
# - Configurable settings: Allows runtime parameter modification
# - Memory activation: Enables context preservation for follow-ups

# Expected Agent Behavior:
# 1. Recognizes query requires current sports schedule information
# 2. Selects Tavily search tool automatically
# 3. Executes search with relevant keywords
# 4. Processes search results to find schedule details
# 5. Formats response with dates, venues, and relevant details

In [ ]:
# Demonstrate Agent Memory with Follow-up Query

# Ask a follow-up question that relies on context from the previous interaction
# This tests the agent's ability to maintain conversation continuity
followup_response = agent1.invoke(
    {"input": "Which teams are participating in that tournament?"},
    config={"configurable": {"session_id": "session1"}}
)

print("\nFollow-up Response:")
print(followup_response['output'])

# Memory Demonstration:
# - "that tournament" refers to the T20 World Cup from previous query
# - Agent maintains context without re-asking for clarification
# - May reuse information from previous search or perform new search
# - Shows natural conversation flow with contextual understanding

## Agent Behavior Analysis

### Key Agent Capabilities Demonstrated
1. **Autonomous Tool Selection**: Agent automatically chooses appropriate tools based on query analysis
2. **Dynamic Reasoning**: Makes decisions about when and how to use external information
3. **Context Preservation**: Maintains conversation history for natural follow-up interactions
4. **Information Synthesis**: Combines tool outputs with reasoning to generate coherent responses
5. **Error Resilience**: Handles tool failures and provides meaningful responses

### Agent vs Traditional Chains
| Feature | Traditional Chains | LangChain Agents |
|---------|-------------------|------------------|
| Execution Path | Predetermined sequence | Dynamic decision-making |
| Tool Usage | Fixed tool calls | Intelligent tool selection |
| Flexibility | Limited to designed flow | Adapts to query requirements |
| Complexity | Simple, predictable | Advanced reasoning capabilities |
| Use Cases | Known workflows | Open-ended problem solving |

### Agent Decision-Making Process
1. **Query Understanding**: Parse user input for intent and requirements
2. **Context Analysis**: Consider conversation history and previous interactions
3. **Tool Assessment**: Evaluate available tools for query relevance
4. **Execution Planning**: Determine sequence of tool calls and reasoning steps
5. **Result Integration**: Combine tool outputs with reasoning for final response

## Key Takeaways and Production Considerations

### Agent Architecture Benefits
- **Flexibility**: Handle diverse queries without pre-programming specific flows
- **Extensibility**: Easy to add new tools and capabilities
- **Intelligence**: Reason about complex multi-step problems
- **Context Awareness**: Maintain conversation state and user preferences
- **Scalability**: Support multiple concurrent conversations with session management

### Production Deployment Considerations
- **Performance**: Agents may be slower than chains due to reasoning overhead
- **Cost**: Multiple LLM calls for reasoning and tool selection increase costs
- **Reliability**: More complex execution paths can introduce failure points
- **Monitoring**: Need comprehensive logging for agent decision tracking
- **Safety**: Implement guardrails for tool usage and response generation

### Advanced Agent Patterns
- **Multi-Agent Systems**: Multiple specialized agents working together
- **Tool Composition**: Combining multiple tools for complex workflows
- **Custom Tools**: Building domain-specific tools for business applications
- **Agent Supervision**: Human-in-the-loop patterns for critical decisions
- **Memory Management**: Sophisticated context and long-term memory systems

### Real-World Applications
- **Customer Support**: Intelligent help desk agents with access to knowledge bases
- **Research Assistants**: Academic and business research with web access
- **Data Analysis**: Agents that can query databases and generate insights
- **Content Creation**: Writing assistants with fact-checking capabilities
- **Process Automation**: Intelligent workflow orchestration with decision-making